In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import re
import random
import string
import math
from difflib import SequenceMatcher

# دالة العشوائية
def calc_entropy(text):
    if not text: return 0
    entropy = 0
    for x in range(256):
        p_x = float(text.count(chr(x))) / len(text)
        if p_x > 0: entropy += - p_x * math.log(p_x, 2)
    return entropy

# دالة التشابه
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# دالة التحليل
def get_features(url):
    my_list = []

    # تنظيف الرابط لاستخراج النطاق كامل
    try:
        # يحول http://ww38.gilhub.com/abc الى ww38.gilhub.com
        clean_url = url.split('//')[-1].split('/')[0].split('?')[0]
        # يقسمه الى اجزاء ['ww38', 'gilhub', 'com']
        domain_parts = clean_url.split('.')
    except:
        clean_url = url
        domain_parts = [url]

    # 1. طول الرابط
    my_list.append(len(url))

    # 2. عدد النقاط
    my_list.append(clean_url.count('.'))

    # 3. وجود @
    my_list.append(1 if '@' in url else 0)

    # 4. العشوائية (Entropy) لاطول كلمة في الرابط
    longest_part = max(domain_parts, key=len) if domain_parts else clean_url
    my_list.append(calc_entropy(longest_part))

    # 5. IP Check
    if re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', url):
        my_list.append(1)
    else:
        my_list.append(0)

    # 6. فحص الشرطة (نتجاهلها اذا كانت في الجزء الاخير مثل .com-sa)
    if '-' in clean_url:
        my_list.append(1)
    else:
        my_list.append(0)

    # 7. كلمات حساسة
    bad_words = ['login', 'verify', 'update', 'account', 'secure', 'banking', 'confirm']
    found_bad = 0
    for w in bad_words:
        if w in url.lower():
            found_bad += 1
    my_list.append(found_bad)

    # 8. خوارزمية "صياد التقليد" (Smart Brand Protection)
    # القائمة الذهبية
    top_targets = [
        'google', 'facebook', 'amazon', 'apple', 'microsoft', 'netflix', 'paypal',
        'dropbox', 'instagram', 'twitter', 'linkedin', 'whatsapp',
        'absher', 'moi', 'nafath', 'stc', 'jarir', 'github', 'coursera'
    ]

    is_typosquatting = 0

    # اللوجيك الجديد: نفحص كل جزء في الرابط
    for part in domain_parts:
        # نتجاهل الاجزاء القصيرة  (ww38, com, sa)
        if len(part) < 4:
            continue

        if part in top_targets: # اذا وجدنا الاسم الاصلي (github)
            continue # هذا جيد، لا تسوي شيء

        # اذا لم يكن هو الاصلي، نقارنه بالاهداف
        for target in top_targets:
            ratio = similar(part, target)
            # تشابه عالي (فوق 0.8) لكن ليس تطابق
            if ratio > 0.80 and ratio < 1.0:
                is_typosquatting = 1
                break
        if is_typosquatting: break

    my_list.append(is_typosquatting)

    # 9. نطاقات مشبوهة
    suspicious_tlds = ['.xyz', '.top', '.site', '.vip', '.cc']
    is_sus = 0
    for tld in suspicious_tlds:
        if url.endswith(tld): is_sus = 1
    my_list.append(is_sus)

    return my_list

print("Cell 1 Fixed: Now scanning subdomains (ww38...)! ✅")

Cell 1 Fixed: Now scanning subdomains (ww38...)! ✅


In [19]:
print("1. Generating Simulation Data...")

def get_random_string(length):
    return ''.join(random.choice(string.ascii_lowercase) for i in range(length))

safe_urls = []
phishing_urls = []
top_targets = ['google', 'amazon', 'github', 'absher', 'jarir', 'coursera']

# 1. الروابط الآمنة
# روابط نظيفة
for i in range(3000):
    dom = random.choice(top_targets)
    url = f"https://www.{dom}.com"
    safe_urls.append(url)

# روابط معقدة (مثل جرير)
for i in range(2000):
    dom = random.choice(top_targets)
    slug = f"{get_random_string(5)}-{get_random_string(5)}"
    url = f"https://www.{dom}.com/item/{slug}"
    safe_urls.append(url)

# 2. الروابط الخبيثة
prefixes = ['ww1', 'ww38', 'www-secure', 'web', 'login']
for i in range(2000):
    target = random.choice(top_targets)
    # صناعة اسم مقلد (github -> gilhub)
    fake = list(target)
    idx = random.randint(0, len(target)-1)
    fake[idx] = random.choice(['l', '1', 'e', 'a'])
    fake_target = "".join(fake)

    if fake_target != target:
        prefix = random.choice(prefixes)
        # النتيجة: http://ww38.gilhub.com
        url = f"http://{prefix}.{fake_target}.com"
        phishing_urls.append(url)

# ب) تلاعب عادي
for i in range(2000):
    target = random.choice(top_targets)
    fake = target + "x"
    url = f"http://www.{fake}.com"
    phishing_urls.append(url)

# ج) IP وعشوائية
for i in range(1000):
    url = f"http://{random.randint(1,255)}.{random.randint(1,255)}/login"
    phishing_urls.append(url)

# --- التدريب ---
print("2. Training...")
min_len = min(len(safe_urls), len(phishing_urls))
safe_urls = safe_urls[:min_len]
phishing_urls = phishing_urls[:min_len]

all_urls = safe_urls + phishing_urls
all_labels = [0]*len(safe_urls) + [1]*len(phishing_urls)
combined = list(zip(all_urls, all_labels))
random.shuffle(combined)

final_features = [get_features(u) for u, l in combined]
X = np.array(final_features)
y = np.array([l for u, l in combined])

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X, y)

print(f"✅ Model Retrained! Ready to detect 'ww38' tricks.")

1. Generating Simulation Data...
2. Training...
✅ Model Retrained! Ready to detect 'ww38' tricks.


In [20]:
import requests

print("--- 🛡️ Enterprise AI Phishing Detector ---")
print("⚠️  DISCLAIMER: This is an educational project powered by AI.")
print("⚠️  It provides a prediction based on patterns, NOT a guarantee.")
print("⚠️  Always verify URLs manually before entering sensitive info.")
print("-" * 50)
print("Type 'exit' to quit.\n")

# --- القائمة البيضاء اليدوية ---
TRUSTED_DOMAINS = {
    # 1. المواقع العالمية
    'github.com', 'www.github.com',
    'google.com', 'www.google.com',
    'youtube.com', 'www.youtube.com',
    'facebook.com', 'www.facebook.com',
    'amazon.com', 'www.amazon.com',
    'twitter.com', 'www.twitter.com', 'x.com', 'www.x.com',
    'instagram.com', 'www.instagram.com',
    'linkedin.com', 'www.linkedin.com',
    'netflix.com', 'www.netflix.com',
    'microsoft.com', 'www.microsoft.com',
    'apple.com', 'www.apple.com',
    'whatsapp.com', 'www.whatsapp.com',

    # 2. مواقع الأمن السيبراني والتعليم
    'offsec.com', 'www.offsec.com',
    'kali.org', 'www.kali.org',
    'tuwaiq.edu.sa', 'www.tuwaiq.edu.sa',
    'satr.tuwaiq.edu.sa', # منصة سطر
    'coursera.org', 'www.coursera.org',
    'udemy.com', 'www.udemy.com',

    # 3. المواقع السعودية الحكومية والجامعية
    'absher.sa', 'www.absher.sa',
    'moi.gov.sa', 'www.moi.gov.sa',
    'jarir.com', 'www.jarir.com',
    'stc.com.sa', 'www.stc.com.sa',

    # 4. نطاقات جامعة المجمعة
    'majmaah.edu.sa', 'www.majmaah.edu.sa',
    'mu.edu.sa', 'www.mu.edu.sa',      # النطاق المختصر
    'm.mu.edu.sa',                     # رابط الجوال
    'sis.mu.edu.sa',                   # النظام الأكاديمي
    'edugate.mu.edu.sa'                # البوابة
}

while True:
    u = input("👉 Enter URL: ").strip()

    if u == 'exit': break
    if not u: continue

    # 1. تنظيف الرابط لاستخراج الدومين الأساسي
    try:
        # نزيل البروتوكول
        clean_u = u.replace("https://", "").replace("http://", "")
        # نأخذ الجزء الأول قبل المسار
        domain_check = clean_u.split('/')[0].lower()
        # نزيل البورت
        domain_check = domain_check.split(':')[0]
    except:
        domain_check = u
# الفلاتر
    # 2. الفلتر الأول: القائمة البيضاء (Whitelist Check) ✅
    is_whitelisted = False

    # هذا يسمح بمرور أي نطاق فرعي مثل any.mu.edu.sa
    for trusted in TRUSTED_DOMAINS:
        if domain_check == trusted or domain_check.endswith("." + trusted):
            is_whitelisted = True
            break

    if is_whitelisted:
        print("🔍 Check: Whitelist Database...")
        print(f"✅ RESULT: Safe (Verified Trusted Domain)")
        print("-" * 30)
        continue

    # 3. الفلتر الثاني: الذكاء الاصطناعي
    feats = get_features(u)
    f_array = np.array(feats).reshape(1, -1)
    res = rf_model.predict(f_array)[0]

    print("🔍 Check: AI Deep Analysis...")

    if res == 1:
        print("🚨 RESULT: PHISHING / MALICIOUS")
        print("⚠️  Reason: Suspicious pattern detected (Typosquatting or anomaly).")
    else:
        print("✅ RESULT: Safe Website")
    print("-" * 30)

--- 🛡️ Enterprise AI Phishing Detector ---
Type 'exit' to quit.

👉 Enter URL: https://github.com
🔍 Check: Whitelist Database...
✅ RESULT: Safe (Verified Trusted Domain)
------------------------------
👉 Enter URL: https://www.jarir.com/asus-vivobook-s-14-oled-laptops-648873.html
🔍 Check: Whitelist Database...
✅ RESULT: Safe (Verified Trusted Domain)
------------------------------
👉 Enter URL: https://www.coursera.org/learn/python-basics/ungradedLti/utvs6/values-and-data-types
🔍 Check: Whitelist Database...
✅ RESULT: Safe (Verified Trusted Domain)
------------------------------
👉 Enter URL: http://gilhub.com
🔍 Check: AI Deep Analysis...
🚨 RESULT: PHISHING / MALICIOUS
⚠️  Reason: Suspicious pattern detected (Typosquatting or anomaly).
------------------------------
👉 Enter URL: http://ww38.gilhub.com/?subid1=20251120-1110-0496-bb4c-4c09e2d69ff1
🔍 Check: AI Deep Analysis...
🚨 RESULT: PHISHING / MALICIOUS
⚠️  Reason: Suspicious pattern detected (Typosquatting or anomaly).
------------------